**Importando bibliotecas que serão úteis para o classificador NayveBayes e determinando uma random_seed**

In [1]:

import pandas as pd
import numpy as np
import seaborn as sns
import pickle
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from nltk import bigrams
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report

#Set seed
seed_ = 194
np.random.seed(seed_)

**Carregando a base de dados a ser utilizada**

In [ ]:

df = pd.read_csv('datasets/df_sem_duplicatas.csv')

**Set-Up do LabelEncoder model**

In [ ]:
#Criação do LabelEncoder
pkl_file = open('models/label_encoder.pkl', 'rb')
le = pickle.load(pkl_file) 
pkl_file.close()

#O código abaixo é para aplicar na coluna categoria o LabelEncoder
le.fit(df['categoria'])
df['categoria'] = le.transform(df['categoria'])

#Atribuição das classes do LabelEncoder a variável target_names
target_names = le.classes_

**Separando em treino e teste e vetorizando por bigram**

In [ ]:
# Separa em treino e teste
X = df['descricao']
y = df['categoria']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = seed_)

# Realiza contagem de token com bigram
vectorizer = CountVectorizer(ngram_range = (2, 2))
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

**Set up Modelo NaiveBayes**

Impressão de metricas e treinamento com o GridSearchCV

In [ ]:
#a função report_modelo é para imprimir as métricas do modelo
def report_modelo(modelo, y, pred):
    print("Acurácia: %f" % accuracy_score(y_true = y, y_pred = pred))
    
    # Plotando a matriz de confusão
    cm = confusion_matrix(y, pred)
    cm = pd.DataFrame(cm, index = target_names, columns = target_names)

    plt.figure(figsize = (8, 8))
    sns.heatmap(cm, cmap = "Blues", linecolor = 'black', linewidths = 1, annot = True, \
                fmt = '', xticklabels = target_names, yticklabels = target_names)
    plt.show()
    
    # Imprimindo métricas
    print(classification_report(y_true = y, y_pred = pred, target_names= target_names))

In [ ]:
#A função treinaClassificacao_GridSearchCV é para treinar o modelo com o GridSearchCV
def treinaClassificacao_GridSearchCV(modelo, params_, x_treino, y_treino, x_teste, y_teste,\
                        n_jobs = -1, cv = 5, refit = True, scoring = None, salvar_resultados = False,\
                       report_treino = False, retorna_modelo = False):
    
    # Realizando treinamento baseado no grid informado
    grid = GridSearchCV(modelo, params_, n_jobs = n_jobs, cv = cv, refit = refit, scoring = scoring)
    
    grid.fit(x_treino, y_treino)
    pred = grid.predict(x_teste)
    modelo_ = grid.best_estimator_

    print(grid.best_params_)
    
    # Realizando previsões
    y_test_pred = modelo_.predict(x_teste)
    
    # Gerando report do modelo
    report_modelo(modelo_, y_teste, y_test_pred) 
    
    # Salvando resultados em dataframe caso solicitado
    if salvar_resultados:
        resultados_df = pd.DataFrame(grid.cv_results_)
        
        if retorna_modelo:
            return resultados_df, modelo_
        else:
            resultados_df
        
    if retorna_modelo:
        return modelo_

In [ ]:
#Instancia modelo base
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_vec, y_train)

**Treinando com o Naive Bayes**

In [ ]:
# Realiza previsões e imprime report
y_pred = nb_classifier.predict(X_test_vec)
report_modelo(nb_classifier, y_test, y_pred) 

**Tunning do NaiveBayes bigram**

In [ ]:
%%time

params = {
    'alpha': [i * 0.001 for i in range(1000)],
    'fit_prior': [True, False]
}

# Tuning Naibe Nayes
modelo_nb = MultinomialNB()

modelo_nb = treinaClassificacao_GridSearchCV(modelo_nb, params, X_train_vec, y_train,\
                                              X_test_vec, y_test)

In [ ]:
# Salvando modelo como pickle
with open('models/naive_bayes_bigram.pkl', 'wb') as model_file:
    pickle.dump(nb_classifier, model_file)

**Vetorizando por Trigram**

In [ ]:
# Realiza contagem de token com trigram
vectorizer = CountVectorizer(ngram_range = (3, 3))
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

**Modelo Base Naive Bayes Trigram**

In [ ]:
# Instancia modelo base
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_vec, y_train)

In [ ]:
# Realiza previsões e imprime report
y_pred = nb_classifier.predict(X_test_vec)
report_modelo(nb_classifier, y_test, y_pred) 

**Tunning Naive Bayes Trigram**

In [ ]:
%%time

params = {
    'alpha': [i * 0.001 for i in range(1000)],
    'fit_prior': [True, False]
}

# Tuning Naibe Nayes
modelo_nb = MultinomialNB()

modelo_nb = treinaClassificacao_GridSearchCV(modelo_nb, params, X_train_vec, y_train,\
                                              X_test_vec, y_test)

In [ ]:
# Salvando modelo como pickle
with open('models/naive_bayes_trigram.pkl', 'wb') as model_file:
    pickle.dump(nb_classifier, model_file)

**Vetorizando por Texto**

In [ ]:
# Realiza contagem de token com trigram
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

**Modelo Base Naive Bayes**

In [ ]:
# Instancia modelo base
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_vec, y_train)

In [ ]:
# Realiza previsões e imprime report
y_pred = nb_classifier.predict(X_test_vec)
report_modelo(nb_classifier, y_test, y_pred) 

**Tunning Naive Bayes vetorizado por Texto**

In [ ]:
%%time

params = {
    'alpha': [i * 0.001 for i in range(1000)],
    'fit_prior': [True, False]
}

# Tuning Naibe Nayes
modelo_nb = MultinomialNB()

modelo_nb = treinaClassificacao_GridSearchCV(modelo_nb, params, X_train_vec, y_train,\
                                              X_test_vec, y_test)

In [ ]:
# Salvando modelo como pickle
with open('models/naive_bayes.pkl', 'wb') as model_file:
    pickle.dump(nb_classifier, model_file)

In [ ]:
# Salvando count vectorizer como pickle
with open('models/countvectorizer_naive_bayes.pkl', 'wb') as model_file:
    pickle.dump(vectorizer, model_file)